### test데이터에서만 class단위로 json에서 data를 extract하는 프로그램

In [ ]:
import json
import os
import pandas as pd
from collections import defaultdict
from tqdm import tqdm

def load_json(json_path):
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        return data
    except Exception as e:
        print(f"JSON 파일을 로드하는 중 오류 발생: {json_path}\n오류 내용: {e}")
        return None

def save_json(data, output_path):
    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=4, ensure_ascii=False)
        print(f"JSON 파일이 '{output_path}' 경로에 저장되었습니다.")
    except Exception as e:
        print(f"JSON 파일을 저장하는 중 오류 발생: {output_path}\n오류 내용: {e}")

def filter_and_split(audio_features, domain='test'):
    split_data = defaultdict(list)

    for entry in tqdm(audio_features.get('audio_features', []), desc=f"Filtering for domain '{domain}'"):
        entry_domain = entry.get('domain', '').lower()
        if entry_domain != domain.lower():
            continue

        machine_type = entry.get('machine_type', 'Unknown').strip()
        split_data[machine_type].append(entry)

    return split_data

# 설정
input_json_path = 'audio_features_merged.json'  # 실제 파일 경로로 수정하세요
domain_to_filter = 'test'
output_dir = 'filtered_jsons_test'  # 원하는 출력 디렉토리 이름으로 수정하세요

# 처리
if not os.path.exists(input_json_path):
    print(f"입력 JSON 파일을 찾을 수 없습니다: {input_json_path}")
else:
    audio_features_data = load_json(input_json_path)

    if audio_features_data:
        split_data = filter_and_split(audio_features_data, domain=domain_to_filter)
        os.makedirs(output_dir, exist_ok=True)

        for machine_type, entries in split_data.items():
            safe_machine_type = ''.join(c if c.isalnum() or c in (' ', '_') else '_' for c in machine_type).replace(' ', '_')
            output_json = os.path.join(output_dir, f'audio_features_{domain_to_filter}_{safe_machine_type}.json')
            save_json({"audio_features": entries}, output_json)

        print(f"모든 JSON 파일이 '{output_dir}' 디렉토리에 저장되었습니다.")


usage: ipykernel_launcher.py [-h] --json JSON [--domain DOMAIN]
                             [--output_dir OUTPUT_DIR]
ipykernel_launcher.py: error: the following arguments are required: --json


SystemExit: 2